<a href="https://colab.research.google.com/github/timsetsfire/wandb-examples/blob/main/colab%5CW%26B_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch + W&B

The purpose of this lab is to instrument W&B a top of existing ML Workflows which might be leveraging 
* PyTorch
* Tensorboard (for metric tracking)
* Python `logging` (for metric tracking)

We will augment this workflow by leveraging 
* Wandb Experiments and syncing with Tensorboard
* Wandb logging
* Wandb Artifacts for dataset and model logging / versioning
* Tables to surface prediction examples on Test datasets
* track lineage of all artifacts and experiments completed

Lastly, we'll do a simple demo of sweeps and interact with the runs via W&B API to
* query runs and run summaries
* artifacts

In [1]:
%%capture
!pip install wandb easydict --upgrade

In [2]:
%%capture
!pip install tensorboard dill

## Logging In

In [26]:
#@title Enter host address
#@markdown Enter the host url which corresponds to your WB instance.
host = "https://api.wandb.ai" #@param {type: "string"}


In [11]:
import wandb
## when using wandb anywhere other than wandb.ai, you must 
## provide a proper host, so the client knows where to communcate
## details of the experiment
# wandb.login(key = key, host = host)
wandb.login(host = host)

True

In [12]:
import os
import random
import logging
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm.notebook import tqdm
from torch.utils.tensorboard import SummaryWriter

# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)

# Device configuration
# if you wind up with any device other than cpu, some code below will need to 
# change specific to the way we are interacting with torch tensors.  
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

# remove slow mirror from list of MNIST mirrors
torchvision.datasets.MNIST.mirrors = [mirror for mirror in torchvision.datasets.MNIST.mirrors
                                      if not mirror.startswith("http://yann.lecun.com")]

## Get Data (and log it)

When we get data and log it, there are obviously tons of way to complete this.  Depending on how you log data, and whether or not you log your retrieval mechanism is a matter of preference and internal guidelines you need to follow.  

In our approach, we will write a `getter` for our data.  The benefit of writing a getting, we can log this getter with our dataset as part of the artifact metadata.



Before we get started it is important to set the name space for your project.  This is going to be accomplished by passing a `project_name` as well as an `entity` to your wandb experiment

`entity` corresponds to the team to which the project will be associated with.  The `entity` could be a team name, or your user name.  

In [13]:
project_name = "wandb-training" #@param {type: "string"}
entity = "tim-w" #@param {type: "string"}

## Logging data

W&B is very unopinionated with regard to how you track your experiments.  We could log data in any number of ways.  
* Log one artifact which represents all the data - training, validation, and test data to one artifact 
* Log several artifacts - one for each of the training, validation, and test data loaders.  

It is a matter of what best suites your needs and workflows and expectations.  

### Anatomy of an artifact 

The `Artifact` class will correspond to an entry in the W&B Artifact registry.  The artifact has 
* a name
* a type
* metadata
* description
* files, directory of files, or references

Example usage 
```
run = wandb.init(project = "my-project")
artifact = wandb.Artifact(name = "my_artifact", type = "data")
artifact.add_file("/path/to/my/file.txt")
run.log_artifact(artifact)
run.finish()
```

In [14]:
## create the data directory locally if it does not already exists
from pathlib import Path
data_path = Path("./data")
data_path.mkdir(exist_ok = True)

## define out data getter 
def get_data(slice=5, train=True):
  '''
  helper function to get data
  args: 
    slice: Int => passed to torch.utils.data.Subset indices argument
    train: Boolean => True to download training data, False for test data
  '''
  full_dataset = torchvision.datasets.MNIST(root=".",
                                            train=train, 
                                            transform=transforms.ToTensor(),
                                            download=True)
  #  equiv to slicing with [::slice] 
  sub_dataset = torch.utils.data.Subset(
    full_dataset, indices=range(0, len(full_dataset), slice))

  return sub_dataset

In [15]:
logging.basicConfig(
                format="%(levelname)s - %(asctime)s - %(message)s",
        )
logger = logging.getLogger("CNN-Logger")
logger.setLevel("INFO")

## Our First W&B Experiment / Run

We are going to 
* get our training and test data
* split the training data into training and validation
* create artifacts for all three dataset
* log those artifacts to W&B.  

In [16]:
#%%wandb -h 600 
import pickle
from dill.source import getsource
from dill import detect
from datetime import datetime 

with wandb.init(project = project_name, job_type = "data-acquisition") as run:

  train, test = get_data(train=True), get_data(train=False)
  train, validation = torch.utils.data.random_split(train, [10000, 2000])

  torch.save(train, './data/training_data.pt')
  torch.save(validation, './data/validation_data.pt')
  torch.save(test, './data/test_data.pt')

  train_artifact = wandb.Artifact(name = "mnist-training-data", type = "dataset", 
                                  description = "training data",
                                  metadata = { 
                                      "data-set": "MNIST training",
                                      "getter": getsource(detect.code(get_data))}
                                  )
  train_artifact.add_file("./data/training_data.pt")

  validation_artifact = wandb.Artifact(name = "mnist-validation-data", type = "dataset", 
                                       description = "validation data",
                                       metadata = { 
                                      "data-set": "MNIST validation",
                                      "getter": getsource(detect.code(get_data))})
  validation_artifact.add_file("./data/validation_data.pt")

  test_artifact = wandb.Artifact(name = "mnist-test-data", type = "dataset", 
                                 description = "test data",
                                 metadata = { 
                                      "data-set": "MNIST test",
                                      "getter": getsource(detect.code(get_data))})
  test_artifact.add_file("./data/test_data.pt")  
  
  run.log_artifact(train_artifact)
  run.log_artifact(validation_artifact)
  run.log_artifact(test_artifact)

## Artifact usage (Creating the DAG)

Part of the value of W&B is the ability to capture lineage via Experiments and Artifacts.  Next up for our work flow is to specifiy a model and commence training.  

It is key to remember that experiments create and consume artifacts and we have already completed one  experimemtns where we created dataset artifacts.  

Next up, we will commence an experiment that will consume the artifacts from the previous run for the purposes of training model, then we will create a model artifact.

## Specify the model



In [17]:
# Conventional and convolutional neural network
class ConvNet(nn.Module):
    def __init__(self, kernels, classes=10):
        super(ConvNet, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, kernels[0], kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, kernels[1], kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7 * 7 * kernels[-1], classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [18]:
def make_loader(dataset, batch_size):
    loader = torch.utils.data.DataLoader(dataset=dataset,
                                         batch_size=batch_size, 
                                         shuffle=True,
                                         pin_memory=True, num_workers=2)
    return loader

## Training

In our first model training experiment, we are going to sync our wandb experiment to tensorboard -> so no wandb specific logging will be instrumented.  


In [19]:
# %%wandb -h 600
# Run training and track with wandb, but no explicit logging.  
# since we were alredy using tensorboard via WritterSumamry, we'll 
# sync w&b to tensorboard.
config = dict(
    epochs=5,
    classes=10,
    kernels=[16, 32],
    batch_size=128,
    learning_rate=0.01,
    dataset="MNIST",
    architecture="CNN"
    )

with wandb.init(project = project_name, 
                 job_type = "training", 
                 config = config,
                 sync_tensorboard = True) as run:

  config = wandb.config
  ## or, ifyou have a nasty nested dictionary for your config
  # config = EasyDict(wandb.config)

  run.use_artifact(f"{run.entity}/{run.project}/mnist-training-data:latest")
  run.use_artifact(f"{run.entity}/{run.project}/mnist-validation-data:latest")
  ## download and instantiation of the artifacts might be necessary.  

  train_loader = make_loader(train, batch_size=config.batch_size)
  validation_loader = make_loader(validation, batch_size=config.batch_size)

  model = ConvNet(config.kernels, config.classes).to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)

  writer = SummaryWriter(log_dir = "./wandb/latest-run")
  total_batches = len(train_loader) * config.epochs
  example_ct = 0  # number of examples seen
  batch_ct = 0
  for epoch in tqdm(range(config.epochs)):
    for step, (images, labels) in enumerate(train_loader):
      images, labels = images.to(device), labels.to(device)
      # Forward pass ➡
      outputs = model(images)
      loss = criterion(outputs, labels)
      # Backward pass ⬅
      optimizer.zero_grad()
      loss.backward()
      # Step with optimizer
      optimizer.step()
      example_ct +=  len(images)
      batch_ct += 1
      # Report metrics every 25th batch
      if ((batch_ct + 1) % 25) == 0:
        writer.add_scalar("Train Metrics/loss", loss, batch_ct)
        writer.add_scalar("epoch", loss, batch_ct)
        logger.info(f"Epoch: {epoch}, Loss: {loss.detach().numpy()}")
    with torch.no_grad():
      correct, total = 0, 0
      for images, labels in validation_loader:
          images, labels = images.to(device), labels.to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
          loss = criterion(outputs, labels)
          writer.add_scalar("Validation Metrics/loss", loss, batch_ct)
          writer.add_scalar("epoch", epoch, batch_ct)
      logger.info(f"Epoch {epoch}, Accuracy of the model on the {total} test images: {100 * correct / total}%")
      writer.add_scalar("Validation Metrics/accuracy", correct/total, batch_ct)
      writer.add_scalar("epoch", epoch, batch_ct)

  torch.save(model.state_dict(), "model.pt")
  model_artifact = wandb.Artifact(name = "mnist-model", type = "model")
  model_artifact.add_file("model.pt")
  run.log_artifact(model_artifact)


  0%|          | 0/5 [00:00<?, ?it/s]

INFO:CNN-Logger:Epoch: 0, Loss: 0.34940022230148315
INFO:CNN-Logger:Epoch: 0, Loss: 0.3246783912181854
INFO:CNN-Logger:Epoch: 0, Loss: 0.1811293363571167
INFO:CNN-Logger:Epoch 0, Accuracy of the model on the 2000 test images: 94.45%
INFO:CNN-Logger:Epoch: 1, Loss: 0.18940120935440063
INFO:CNN-Logger:Epoch: 1, Loss: 0.07393933087587357
INFO:CNN-Logger:Epoch: 1, Loss: 0.10079780966043472
INFO:CNN-Logger:Epoch 1, Accuracy of the model on the 2000 test images: 96.65%
INFO:CNN-Logger:Epoch: 2, Loss: 0.028136445209383965
INFO:CNN-Logger:Epoch: 2, Loss: 0.08603929728269577
INFO:CNN-Logger:Epoch: 2, Loss: 0.04594190791249275
INFO:CNN-Logger:Epoch 2, Accuracy of the model on the 2000 test images: 97.5%
INFO:CNN-Logger:Epoch: 3, Loss: 0.03457905352115631
INFO:CNN-Logger:Epoch: 3, Loss: 0.0681949034333229
INFO:CNN-Logger:Epoch: 3, Loss: 0.06608622521162033
INFO:CNN-Logger:Epoch 3, Accuracy of the model on the 2000 test images: 97.5%
INFO:CNN-Logger:Epoch: 4, Loss: 0.10246636718511581
INFO:CNN-Log

Train Metrics/loss,█▇▄▅▂▃▁▂▁▁▂▂
Validation Metrics/accuracy,▁▆█
Validation Metrics/loss,█▂▁▄
epoch,▂▂▁▁▁▁▁▃▁▁▁▆▁▁▁█
global_step,▁▂▂▂▃▃▄▄▅▅▆▆▆▇██
Train Metrics/loss,0.06609
Validation Metrics/accuracy,0.975
Validation Metrics/loss,0.13811
epoch,3.0
global_step,316


## Test Data Evaluation

In [20]:
import pandas as pd
with wandb.init(project = project_name, entity = entity, job_type = "evaluation") as run:
  model_artifact = run.use_artifact(model_artifact.wait())
  ## instantiate the model if necessary
  # model_dir = model_artifact.download()
  # model = ConvNet(config.kernels, config.classes)
  # model.load_state_dict(torch.load(f"{model_dir}/model.pt"))
  run.use_artifact(f"{run.entity}/{run.project}/mnist-test-data:latest")
  ## same goes for the dataset
  test_loader = make_loader(test, batch_size=config.batch_size)

  model.eval()
  # Run the model on some test examples

  with torch.no_grad():
      correct, total = 0, 0
      total_loss = 0
      all_data = []
      for images, labels in test_loader:
          images, labels = images.to(device), labels.to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
          loss = criterion(outputs, labels)*labels.size(0)
          total_loss += loss
          wandb_images = []
          for image in images.numpy():
            temp = wandb.Image(image)
            wandb_images.append(temp) 
          scores = pd.DataFrame( outputs.numpy().tolist(), columns = [f"p{i}" for i in range(outputs.shape[1])]).to_dict(orient = "series")
          data = {"images":wandb_images, "predicted": predicted.numpy().tolist(), "labels": labels.numpy().tolist()}
          data = {**data, **scores}
          all_data.append(pd.DataFrame(data))
      import pandas as pd 
      df = pd.concat(all_data)
      wandb.log({"Predictions vs Actuals": wandb.Table(dataframe = df)})
      run.log({"Test Metrics/loss": total_loss / total, "Test Metrics/accuracy": correct / total})
      logger.info(f"Accuracy of the model on the {total} " +
            f"test images: {100 * correct / total}%")
          

INFO:CNN-Logger:Accuracy of the model on the 2000 test images: 97.25%


Test Metrics/accuracy,▁
Test Metrics/loss,▁
Test Metrics/accuracy,0.9725
Test Metrics/loss,0.09627


## Sweeps

Very Naive instrumentation of sweeps - sweeps running a single machine

We'll switch from using the `sync_tensorboard` argument to using `wandb.log`.  

Also, we'll use sweeps to evaluate different batch sizes and learning rates.  

1. Add wandb: In your Python script, add a couple lines of code to log hyperparameters and output metrics from your script.
2. Write config: Define the variables and ranges to sweep over. Pick a search strategy— we support grid, random, and Bayesian search, plus techniques for faster iterations like early stopping. Check out some example configs here.
Initialize sweep: Launch the sweep server. We host this central controller and coordinate between the agents that execute the sweep.
3. Launch agent(s): Run a single-line command on each machine you'd like to use to train models in the sweep. The agents ask the central sweep server what hyperparameters to try next, and then they execute the runs.
4. Visualize results: Open our live dashboard to see all your results in one central place.

![](https://1039519455-files.gitbook.io/~/files/v0/b/gitbook-x-prod.appspot.com/o/spaces%2F-Lqya5RvLedGEWPhtkjU-1972196547%2Fuploads%2Fgit-blob-d7820a5646e118213a46afd4faa2c02eed7faf5c%2Fcentral-sweep-server-3%20(2)%20(2)%20(3)%20(3)%20(2)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(3)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(1)%20(2).png?alt=media)

## Sweep Configuration

In [21]:
sweep_config = {
  'method': 'grid', 
  'metric': {
      'name': 'Validation Metrics/loss',  ## matches what i write via SummaryWriter
      'goal': 'minimize'
  },
  'early_terminate':{
      'type': 'hyperband',
      'min_iter': 5
  },
  'parameters': {
      'learning_rate':{
          'values': [0.05,0.025,0.01,0.005,0.001]
      }, 
      'batch_size': { 
          'values': [128, 256]
      }
  }
}


## Training Function

In [22]:
def sweep_train(config_defaults = dict(learning_rate=0.01, batch_size = 128)): 

  config_standard = dict(
    epochs=5,
    classes=10,
    kernels=[16, 32],
    batch_size=128,
    dataset="MNIST",
    architecture="CNN"
    )
  
  config = {**config_defaults, **config_standard}

  with wandb.init(config = config, sync_tensorboard = False) as run:

    config = wandb.config
    ## or, ifyou have a nasty nested dictionary for your config
    # config = EasyDict(wandb.config)

    run.use_artifact(f"{run.entity}/{run.project}/mnist-training-data:latest")
    run.use_artifact(f"{run.entity}/{run.project}/mnist-validation-data:latest")

    train_loader = make_loader(train, batch_size=config.batch_size)
    validation_loader = make_loader(validation, batch_size=config.batch_size)


    model = ConvNet(config.kernels, config.classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)

    total_batches = len(train_loader) * config.epochs
    example_ct = 0  # number of examples seen
    batch_ct = 0
    for epoch in tqdm(range(config.epochs)):
      for _, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        # Forward pass ➡
        outputs = model(images)
        loss = criterion(outputs, labels)
        # Backward pass ⬅
        optimizer.zero_grad()
        loss.backward()
        # Step with optimizer
        optimizer.step()
        example_ct +=  len(images)
        batch_ct += 1
        # Report metrics every 25th batch
        if ((batch_ct + 1) % 25) == 0:
          logger.info(f"Epoch: {epoch}, Loss: {loss.detach().numpy()}")
          run.log({ "Train Metrics/loss": loss, "epoch": epoch})
      with torch.no_grad():
        correct, total = 0, 0
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            loss = criterion(outputs, labels)
            run.log({"Validation Metrics/loss": loss, "epoch": epoch})
        logger.info(f"Epoch {epoch}, Accuracy of the model on the {total} validation images: {100 * correct / total}%")
        run.log({"Validation Metrics/accuracy": correct / total, "epoch": epoch})

    torch.save(model.state_dict(), f"{run.id}-model.pt")
    model_artifact = wandb.Artifact(name = f"{run.id}-mnist-model", type = "model")
    model_artifact.add_file(f"{run.id}-model.pt")
    run.log_artifact(model_artifact)


## Initiate the Sweep

In [23]:
sweep_id = wandb.sweep(sweep_config, project=project_name)

Create sweep with ID: hg5xgkx5
Sweep URL: https://wandb.ai/tim-w/wandb-training/sweeps/hg5xgkx5


## Run the Sweep Agent

In [24]:
wandb_agent = wandb.agent(sweep_id, function=sweep_train, count = 5)

wandb: Agent Starting Run: 6kuo9516 with config:
wandb: 	batch_size: 128
wandb: 	learning_rate: 0.05


  0%|          | 0/5 [00:00<?, ?it/s]

INFO:CNN-Logger:Epoch: 0, Loss: 0.7944325804710388
INFO:CNN-Logger:Epoch: 0, Loss: 0.1905856430530548
INFO:CNN-Logger:Epoch: 0, Loss: 0.1051725223660469
INFO:CNN-Logger:Epoch 0, Accuracy of the model on the 2000 validation images: 92.7%
INFO:CNN-Logger:Epoch: 1, Loss: 0.260639488697052
INFO:CNN-Logger:Epoch: 1, Loss: 0.21069897711277008
INFO:CNN-Logger:Epoch: 1, Loss: 0.20973238348960876
INFO:CNN-Logger:Epoch 1, Accuracy of the model on the 2000 validation images: 94.25%
INFO:CNN-Logger:Epoch: 2, Loss: 0.24138860404491425
INFO:CNN-Logger:Epoch: 2, Loss: 0.20259487628936768
INFO:CNN-Logger:Epoch: 2, Loss: 0.11992068588733673
INFO:CNN-Logger:Epoch 2, Accuracy of the model on the 2000 validation images: 96.2%
INFO:CNN-Logger:Epoch: 3, Loss: 0.12849971652030945
INFO:CNN-Logger:Epoch: 3, Loss: 0.05935431271791458
INFO:CNN-Logger:Epoch: 3, Loss: 0.1654828041791916
INFO:CNN-Logger:Epoch 3, Accuracy of the model on the 2000 validation images: 95.05%
INFO:CNN-Logger:Epoch: 4, Loss: 0.0264818426

Train Metrics/loss,█▃▂▃▃▃▃▃▂▂▁▂▁▁▂
Validation Metrics/accuracy,▁▄█▆▇
Validation Metrics/loss,▅▃▆▅▄▃▃▅▄▅▃▄▃▃▄▇▃▁█▂▂▂▃▄▁▅▂▂▅▄▁█▄▃▂▂▂▃▃▂
epoch,▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆████████
Train Metrics/loss,0.1332
Validation Metrics/accuracy,0.958
Validation Metrics/loss,0.09933
epoch,4


wandb: Agent Starting Run: vwpwihe9 with config:
wandb: 	batch_size: 128
wandb: 	learning_rate: 0.025


...

Train Metrics/loss,█▅▄▃▃▃▆▂▂▁▄▂▂▁▁
Validation Metrics/accuracy,▁▃█▇█
Validation Metrics/loss,▄▅▆▆▅▇█▅▅▅▅▄▅▃▄▇▂▄▂▂▆▁▂▅▄▃▃▃▃▃▄▃▃▆▃▁▂▃▁▂
epoch,▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆████████
Train Metrics/loss,0.02705
Validation Metrics/accuracy,0.978
Validation Metrics/loss,0.03561
epoch,4


wandb: Agent Starting Run: dio7vunj with config:
wandb: 	batch_size: 128
wandb: 	learning_rate: 0.001


  0%|          | 0/5 [00:00<?, ?it/s]

INFO:CNN-Logger:Epoch: 0, Loss: 0.9461560249328613
INFO:CNN-Logger:Epoch: 0, Loss: 0.47701188921928406
INFO:CNN-Logger:Epoch: 0, Loss: 0.2675909399986267
INFO:CNN-Logger:Epoch 0, Accuracy of the model on the 2000 validation images: 91.45%
INFO:CNN-Logger:Epoch: 1, Loss: 0.2856113016605377
INFO:CNN-Logger:Epoch: 1, Loss: 0.25610169768333435
INFO:CNN-Logger:Epoch: 1, Loss: 0.15714676678180695
INFO:CNN-Logger:Epoch 1, Accuracy of the model on the 2000 validation images: 95.0%
INFO:CNN-Logger:Epoch: 2, Loss: 0.20758090913295746
INFO:CNN-Logger:Epoch: 2, Loss: 0.07234368473291397
INFO:CNN-Logger:Epoch: 2, Loss: 0.07864610850811005
INFO:CNN-Logger:Epoch 2, Accuracy of the model on the 2000 validation images: 96.05%
INFO:CNN-Logger:Epoch: 3, Loss: 0.11934518069028854
INFO:CNN-Logger:Epoch: 3, Loss: 0.09787408262491226
INFO:CNN-Logger:Epoch: 3, Loss: 0.0688934177160263
INFO:CNN-Logger:Epoch 3, Accuracy of the model on the 2000 validation images: 96.2%
INFO:CNN-Logger:Epoch: 4, Loss: 0.12277697

Train Metrics/loss,█▄▃▃▃▂▂▁▁▂▂▁▂▂▁
Validation Metrics/accuracy,▁▅▇▇█
Validation Metrics/loss,▇▇▅█▅▆▅▅▃▄▃▂▄▄▄▄▂▃▂▂▃▂▄▄▂▂▂▁▄▄▁▃▁▃▂▂▂▃▂▂
epoch,▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆████████
Train Metrics/loss,0.03239
Validation Metrics/accuracy,0.9715
Validation Metrics/loss,0.07068
epoch,4


## Use API to interact with W&B

In [25]:
import pandas as pd
import wandb
api = wandb.Api()
sweep = api.sweep(f"{entity}/{project_name}/{sweep_id}")
temp_data = []
for r in sweep.runs:
 temp_dict = dict(**dict(r.summary), **r.config)
 temp_dict["run_id"] = r.id
 temp_dict["run_name"] = r.name
 temp_data.append( temp_dict)
df = pd.DataFrame(temp_data)
df.set_index("run_id", inplace = True)
best_run_id = sweep.best_run().id
best_run = api.run(f"{entity}/{project_name}/{best_run_id}")
df.loc[best_run_id]

wandb: Sorting runs by +summary_metrics.Validation Metrics/loss


_step                                         99
epoch                                          4
_wandb                           {'runtime': 40}
_runtime                               40.538221
_timestamp                     1660166801.760658
Train Metrics/loss                      0.027049
Validation Metrics/loss                 0.035614
Validation Metrics/accuracy                0.978
epochs                                         5
classes                                       10
dataset                                    MNIST
kernels                                 [16, 32]
batch_size                                   128
architecture                                 CNN
learning_rate                              0.005
run_name                           fresh-sweep-4
Name: b2wqqskj, dtype: object